# Pre-compile LightGBM GPU version to save GPU quota
why create this kernel?
when you want use LigthGMB GPU version, as this thread(https://www.kaggle.com/kirankunapuli/ieee-fraud-lightgbm-with-gpu/comments#LightGBM-GPU-Installation) posted. you have to download and compile LightGBM every time when you started the kernel. comiling is time-wasting operation, so we can compile it well in one kernel and use it directly in other kernels without compiling it again.
how to use it:
* add this kernel's output to your data source;
* do the following codes in your kernel:

> !rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
> 
> !apt-get install -y -qq libboost-all-dev

> %%bash
> 
> cd /tmp
> 
> tar -xvf /kaggle/input/compile-lgbm/LightGBM.tar > /dev/null

> !cd /tmp/LightGBM/python-package/;python3 setup.py install --precompile
> 
> !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
> 
> !pip install -q 'pandas==0.25' --force-reinstall

there'a another time-wasting step: install libboost-all-dev (in your kernel.). but it seems no way to save time in this step, coz installation alone cost about 40s. if anyone have some idea about this please let me know.

Upvote if you like it, thank you!

# install libboost, compiling depends on it.

In [ ]:
%%time
!apt-get install -y -qq libboost-all-dev

# compile LightGBM

In [ ]:
%%time
%%bash

cd /tmp
if [ -d "LightGBM" ];then rm -r LightGBM; fi;
git clone --recursive https://github.com/Microsoft/LightGBM

cd ./LightGBM
if [ -d "./build" ];then rm -r build; fi;
mkdir build

cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

# install LightGBM (optional here) to test if compiling is OK.

In [ ]:
!cd /tmp/LightGBM/python-package/;python3 setup.py install --precompile

!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
# !rm -r LightGBM

# Latest Pandas version
!pip install -q 'pandas==0.25' --force-reinstall

# tar the LightGBM folder as kernel output.

In [ ]:
import tarfile

with tarfile.open('./LightGBM.tar', 'w') as tar:
    tar.add('/tmp/LightGBM', arcname='LightGBM')

!du -sh ./*